# Noise Interferometry Test
This tests the functionality of the noise interferometry module of ooipy

## Sections
- Compute Cross Correlation for specified window

First, let's import all dependancies

In [22]:
import os
import sys
cwd = os.getcwd()
ooipy_dir = os.path.dirname(os.path.dirname(cwd))
sys.path.append(ooipy_dir)
import datetime
import numpy as np
from obspy import read,Stream, Trace
from obspy.core import UTCDateTime
from ooipy.hydrophone import Noise_Interferometry

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Compute Cross Correlation for Specified Window

In [23]:
num_periods = 1
avg_time = 10 #minutes
start_time = datetime.datetime(2017,3,10,0,0,0) # time of first sample
node1 = '/LJ01C'
node2 = '/PC01A'
filter_data = True

#Initialize Hydrophone Xcorr Class
hyd_xcorr = Noise_Interferometry.Hydrophone_Xcorr(node1=node1, node2=node2, avg_time=avg_time, mp=True, W=60, filter_data=filter_data)

Distance Between Hydrophones:  38897.93618108771  meters
Estimate Time Delay Between Hydrophones:  26.28238931154575  seconds
Bearing Between Hydrophone 1 and 2:  297.28608345  degrees


In [26]:
# Average Cross Correlation Over Specified time
t, xcorr_stack, bearing_max = hyd_xcorr.avg_over_mult_periods(num_periods, start_time)



Time Period:  1
Getting Audio from Node 1...
Fetching URLs...
Sorting valid URLs for Time Window...
Getting Audio from Node 2...
Fetching URLs...
Sorting valid URLs for Time Window...
Time to Download Data from Server:  36.12439703941345
Filtering Data...
Correlating Data...
Time to Calculate Cross Correlation of 1 period:  9.668826818466187
Time to Complete 1 period:  55.235698223114014
